In [11]:
import pandas as pd
import warnings
import re
import matplotlib.pyplot as plt

In [12]:
df_emosi=pd.read_csv(r'src\Twitter_Emotion_Dataset.csv')

In [13]:
df_pemilu=pd.read_csv(r'src\pemilu-2024.csv', on_bad_lines='skip')

### Missval Check

In [14]:
df_emosi.isna().sum()

label    0
tweet    0
dtype: int64

### Masking

In [15]:
# Fungsi untuk menghitung kata yang dimask dalam sebuah teks
def count_masked_words(text):
    if isinstance(text, str):
        pattern = r'\[USERNAME\]|\[USER\]'  # Pola regex untuk mencari [USERNAME] atau [USER]
        matches = re.findall(pattern, text)
        return len(matches)
    else:
        return 0  # Jika nilai tidak bertipe string, kembalikan 0

# Menerapkan fungsi ke kolom teks dalam DataFrame
df_emosi['masked_word_count'] = df_emosi['tweet'].apply(count_masked_words)

# Jumlah kata yang dimask dalam keseluruhan dataset
total_masked_words = df_emosi['masked_word_count'].sum()

print("Total kata yang dimask dalam dataset:", total_masked_words)

Total kata yang dimask dalam dataset: 1793


In [16]:
# Fungsi untuk menemukan dan menghitung pola yang diawali dengan tanda kurung siku dalam sebuah teks
def find_and_count_patterns(text):
    if isinstance(text, str):
        pattern = r'\[([^]]+)\]'  # Pola regex untuk mencari semua pola yang diawali dengan tanda kurung siku
        matches = re.findall(pattern, text)

        # Menghitung jumlah kemunculan setiap pola
        pattern_counts = {}
        for match in matches:
            if match in pattern_counts:
                pattern_counts[match] += 1
            else:
                pattern_counts[match] = 1

        return pattern_counts
    else:
        return {}  # Jika nilai tidak bertipe string, kembalikan dictionary kosong

# Menerapkan fungsi ke kolom teks dalam DataFrame
df_emosi['pattern_counts'] = df_emosi['tweet'].apply(find_and_count_patterns)

# Menggabungkan hasil dari semua tweet menjadi satu dictionary
all_patterns_counts = {}
for pattern_count in df_emosi['pattern_counts']:
    for pattern, count in pattern_count.items():
        if pattern in all_patterns_counts:
            all_patterns_counts[pattern] += count
        else:
            all_patterns_counts[pattern] = count

In [17]:
# Membuat barplot
all_patterns_counts = sorted(all_patterns_counts.items(), key=lambda x:x[1], reverse=True)
all_patterns_counts = dict(all_patterns_counts)
for k, i in all_patterns_counts.items():
  print(k, i)

USERNAME 1793
URL 621
askmf 10
SENSITIVE-NO 5
askMF 4
Idm 2
AskMF 2
C48 2
idm 1
Seo In Ha, Love Rain 1
Askmf 1
Late Post 1
Allamah Thabathabai 1
Kartu 1 pria thd wanita 1
Habis buka Facebook 1
Thinking.. 1
BELAJARLAH DEMI ORANGTUAMU!!! 1
BB 1
Obrolan dengan dospem 1 & 2 di grup WA menjelang besok sidang 1
Satu menit kemudian 1
1 1


In [18]:
df_emosi.tail()

,label,tweet,masked_word_count,pattern_counts
4396,love,"Tahukah kamu, bahwa saat itu papa memejamkan m...",0,{}
4397,fear,Sulitnya menetapkan Calon Wapresnya Jokowi di ...,0,{}
4398,anger,"5. masa depannya nggak jelas. lha iya, gimana ...",0,{}
4399,happy,[USERNAME] dulu beneran ada mahasiswa Teknik U...,1,{'USERNAME': 1}
4400,sadness,"Ya Allah, hanya Engkau yang mengetahui rasa sa...",0,{}


### Slang dan Abreviasi

In [19]:
kamus_slang=pd.read_csv(r'src\colloquial-indonesian-lexicon.csv')
kamus_slang=kamus_slang.rename(columns = {'slang' : 'kamus_slang' , 'formal' : 'kamus_perbaikan'})

# Rekonstruksi data sebagai 'dict'
slang_mapping = dict(zip(kamus_slang['kamus_slang'], kamus_slang['kamus_perbaikan']))
kamus_singkatan = pd.read_csv(r'src\kamus_singkatan.csv', header=None, names=['sebelum_perbaikan', 'setelah_perbaikan'],delimiter=';')
singkatan_mapping=dict(zip(kamus_singkatan['sebelum_perbaikan'],kamus_singkatan['setelah_perbaikan']))

### Stopword, emoji, dan Stemmer Factory

In [22]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import  StopWordRemoverFactory
import emoji
from spacy.lang.id import Indonesian
import string

In [23]:
stopword_factory = StopWordRemoverFactory()
stopwords = stopword_factory.get_stop_words()
# List of words with negation meaning
emoji_data = emoji.EMOJI_DATA

# Remove negation words from stopwords
# stopwords = set(stopwords).difference(excluded_stopwords)
nlp = Indonesian()
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [24]:
def replace_emoji_with_ascii(text, emoji_data, language='id'):
    for emoji, translations in emoji_data.items():
        if language in translations:
            text = text.replace(emoji, translations[language])
    return text

In [25]:
text_with_emoji = "Saat kamu merenungkan tentang kehilangan yang pernah kamu alami, luka-luka itu terasa kembali dalam ingatan. 💔🌼 #RememberingLoss"
a = replace_emoji_with_ascii(text_with_emoji, emoji_data, language='id')
a = a.replace(":",' ').replace('_','mask').replace('-','rus').strip()
a = re.sub(' +', ' ', a)
print(a)

Saat kamu merenungkan tentang kehilangan yang pernah kamu alami, lukarusluka itu terasa kembali dalam ingatan. patahmaskhati mekar #RememberingLoss


In [26]:
def process_tweet(tweet) :
  tweet=tweet.lower()
  # link
  tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',tweet)

  # spesifik
  tweet = re.sub(r'\[username\]|\[url\]|\[askmf\]|\[sensitive-no\]|\[satu menit kemudian\]|\[c48\]|\[idm\]', '', tweet)

  # emoji
  tweet=replace_emoji_with_ascii(tweet,emoji_data)
  tweet=tweet.replace(":",' ').replace('_','mask').replace('-','rus').strip()
  tweet=re.sub(' +', ' ', tweet)

  # tokenisasi
  tokens = tweet.split()

  tweet_tokens = []
  for ele in tokens:
    ele_kamus = kamus_singkatan.get(ele, ele)
    ele_slang = slang_mapping.get(ele_kamus, ele_kamus)
    tweet_tokens.append(ele_slang)

  tweet = ' '.join(tweet_tokens)
  tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
  tweet = re.sub(r'#([^\s]+)', '', tweet)
  tweet=re.sub(r'\d+', '', tweet)
  tweet = tweet.strip('\'"')
  tweet = tweet.lstrip('\'"')

  tweet = "".join([char for char in tweet if char not in string.punctuation])

  doc = nlp(tweet)

  tokens = [token.text for token in doc]
      # Hapus stopwords dari tokens
  filtered_tokens = [token for token in tokens if token.lower() not in stopwords]
  tweet = ' '.join(filtered_tokens)

  tweet=stemmer.stem(tweet)
  tweet=tweet.replace('mask',' ').replace('rus','-')

  return tweet

In [27]:
print(process_tweet('hai sayangnya adalah 😂'))

hai sayang wajah gembira berurai air mata


In [28]:
print(process_tweet(str(df_emosi['tweet'][0])))

soal jalan jatibarupolisi gertak gubernur emangny polisi ikut pmbhasan jangan politik atur wilayahhak gubernur soal tn abang soal turun temurunpelikperlu sabar


In [29]:
df_emosi.head(5)

,label,tweet,masked_word_count,pattern_counts
0,anger,"Soal jln Jatibaru,polisi tdk bs GERTAK gubernu...",2,"{'USERNAME': 2, 'URL': 1}"
1,anger,"Sesama cewe lho (kayaknya), harusnya bisa lebi...",0,{}
2,happy,Kepingin gudeg mbarek Bu hj. Amad Foto dari go...,0,{}
3,anger,"Jln Jatibaru,bagian dari wilayah Tn Abang.Peng...",0,{'URL': 1}
4,happy,"Sharing pengalaman aja, kemarin jam 18.00 bata...",1,{'USERNAME': 1}


In [31]:
df_emosi['tweet'] = df_emosi['tweet'].apply(lambda x: process_tweet(str(x)))

In [32]:
df_emosi.to_csv(r'src\cleaned.csv')

In [33]:
df_emosi['tweet']

0       soal jalan jatibarupolisi gertak gubernur eman...
1       sama cewek lho kayak ha- lebih rasai lah sibuk...
2       pengin gudeg mbarek bu hj foto google sengaja ...
3       jalan jatibarubagian wilayah tn abangpengatura...
4       sharing alam aja kemarin jam batalin tiket sta...
                              ...                        
4396    tahu kamu papa mejam mata tahan gejolak batin ...
4397    sulit tetap calon wapresnya jokowi pilpres sal...
4398    masa depan enggak jelas lah iya bagaimana mau ...
4399    dulu benar mahasiswa teknik ui tembak pacar pa...
4400       allah engkau tahu rasa sakit hati sembuh allah
Name: tweet, Length: 4401, dtype: object